In [ ]:
library(harmony)
args <- commandArgs(TRUE)
indir<-args[1]
output_dir<-args[2]
prefix <-args[3]

setwd(indir)
scRNAlist = {}
sams=c("C1","C2","C3","C4","follicle","CA","CL","CLR","Background")
for(i in (1:length(sams))){
     counts <- Read10X(data.dir = paste0(indir,"/",sams[i]),  gene.column = 1)
     scRNAlist[[i]]=CreateSeuratObject(counts,project=sams[i])
     scRNAlist[[i]]@meta.data$orig.ident=sams[i]
}

scRNA=scRNAlist[[1]]
for (j in (2:length(scRNAlist))){
    scRNA <- merge(scRNA,scRNAlist[j])
}

saveRDS(scRNA, file =  paste0(output_dir,"/",prefix,".merge.rds"))


scRNA <- NormalizeData(scRNA, normalization.method = "LogNormalize", scale.factor = 10000)
scRNA <- FindVariableFeatures(scRNA, selection.method = "vst", nfeatures = 10000)
scRNA <- PercentageFeatureSet(scRNA, '^MT-', col.name = "percent.mt")
scRNA <- PercentageFeatureSet(scRNA, '^RP[SL]', col.name = "percent.ribo")
scRNA <- PercentageFeatureSet(scRNA, '^HB[^(P)]', col.name = "percent.hb")
scRNA <- ScaleData(scRNA,vars.to.regress =c("percent.mt","nCount_RNA","orig.ident"))
scRNA <- RunPCA(scRNA,features = VariableFeatures(object =scRNA),npcs=50, verbose=FALSE)
scRNA <- RunHarmony(scRNA, group.by.vars="orig.ident", assay.use="RNA", max.iter.harmony = 50)
scRNA <- RunUMAP(scRNA, reduction = "harmony", dims = 1:50)
scRNA <- RunTSNE(scRNA, reduction = "harmony", dims = 1:50)
scRNA <- FindNeighbors(scRNA, reduction="harmony", dims = 1:50)
saveRDS(scRNA,file=paste0(output_dir,"/",prefix,".harmony.rds"))
